In [1]:
import numpy as np
import pandas as pd
import ast
from lightweight_charts import Chart
from lightweight_charts.widgets import StreamlitChart
import json
import streamlit

In [2]:
df = pd.read_csv('F:\ML_projects\Internship1\data\TSLA_data - Sheet1.csv', parse_dates=['timestamp'])

In [3]:
df.head(5)

,timestamp,direction,Support,Resistance,open,high,low,close,volume
0,2022-08-25,SHORT,"[840, 880]","[900, 895, 940]",302.542,302.807,291.556,295.897,6685.7261
1,2022-08-26,SHORT,"[877, 880, 840, 835, 845]","[900, 908, 980.5, 940]",297.297,301.956,287.376,288.017,508.5590
2,2022-08-27,SHORT,[290],"[300, 322, 323, 324, 422]",297.297,301.956,287.376,288.017,508.5590
3,2022-08-28,LONG,"[285, 286, 84]","[290, 292, 295]",297.297,301.956,287.376,288.017,508.5590
4,2022-08-31,SHORT,"[280, 247, 150, 185]","[287, 288, 292]",280.497,280.837,271.776,275.337,94.2867


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665 entries, 0 to 664
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   timestamp   665 non-null    datetime64[ns]
 1   direction   335 non-null    object        
 2   Support     665 non-null    object        
 3   Resistance  665 non-null    object        
 4   open        665 non-null    float64       
 5   high        665 non-null    float64       
 6   low         665 non-null    float64       
 7   close       665 non-null    float64       
 8   volume      665 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 46.9+ KB


In [5]:
df.isna().sum()

timestamp       0
direction     330
Support         0
Resistance      0
open            0
high            0
low             0
close           0
volume          0
dtype: int64

In [6]:
df[["open","high","low","close","volume"]].describe()

,open,high,low,close,volume
count,665.000000,665.000000,665.000000,665.000000,665.000000
mean,233.765603,239.119302,228.496779,233.849032,100.987451
std,65.874333,67.713161,64.107825,65.773434,258.916828
min,102.916000,111.717000,101.200000,110.336000,32.727600
25%,184.967000,186.746000,181.257000,184.956000,67.935600
50%,224.057000,227.746000,218.197000,220.897000,86.149200
75%,259.126000,263.587000,253.096000,260.326000,107.366400
max,466.117000,488.306000,450.996000,460.396000,6685.726100


In [7]:
for col in ("Support", "Resistance"):
    df[f"{col.lower()}_list"] = df[col].apply(ast.literal_eval)
for prefix in ("support", "resistance"):
    low_col = f"{prefix}_low"
    high_col = f"{prefix}_high"
    list_col = f"{prefix}_list"
    df[low_col]  = df[list_col].apply(lambda lst: min(lst) if lst else np.nan)
    df[high_col] = df[list_col].apply(lambda lst: max(lst) if lst else np.nan)

In [11]:
# --- Create Chart ---
chart = Chart()

# Main candlestick series
ohlcv = df.rename(columns={"timestamp": "time"})
ohlcv['time'] = ohlcv['time'].dt.strftime('%Y-%m-%d')  # Convert to string
chart.set(ohlcv[["time", "open", "high", "low", "close", "volume"]])

# Function to add lines
def add_line(series_name: str, col_name: str):
    series = chart.create_line(series_name)
    data = df[["timestamp", col_name]].copy()
    data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d')  # Convert to string
    data = data.rename(columns={"timestamp": "time"})
    series.set(data, format_cols=False)

# Add support/resistance lines
add_line("support_low", "support_low")
add_line("support_high", "support_high")
add_line("resistance_low", "resistance_low")
add_line("resistance_high", "resistance_high")

# Add markers
for _, row in df.iterrows():
    if row.direction == "LONG":
        chart.marker(text="▲", time=row.timestamp.strftime('%Y-%m-%d'), position="belowBar", color="green")
    elif row.direction == "SHORT":
        chart.marker(text="▼", time=row.timestamp.strftime('%Y-%m-%d'), position="aboveBar", color="red")

# Render in Jupyter-friendly way
chart.show(block=False)  # Remove block=True for Jupyter compatibility|

AssertionError: cannot start a process twice

In [ ]:
df.head()